<a href="https://colab.research.google.com/github/praveentecsl/High-Engagement-Content-Classification-model/blob/main/SocialMedia_engagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt

In [ ]:
url = 'https://raw.githubusercontent.com/praveentecsl/High-Engagement-Content-Classification-model/main/ad_campaign_data.csv'
df = pd.read_csv(url)

In [ ]:
print(df.head()) #commit for testing

  user_id  age  gender  location  device_type   ad_id  ad_category  \
0  U25795   45       0         2            2  A77161            2   
1  U10860   29       0         0            2  A41167            1   
2  U86820   59       0         1            2  A55186            1   
3  U64886   47       0         1            2  A59722            4   
4  U16265   36       1         0            0  A38955            2   

   impressions  clicks  conversions  ...  tfidf_11 tfidf_12  tfidf_13  \
0         2460    1223            1  ...       0.0      0.0       0.0   
1          162     117            1  ...       0.0      0.0       0.0   
2         3648     554            1  ...       0.0      0.5       0.5   
3          648     259            1  ...       0.0      0.5       0.5   
4         3598    1860            1  ...       0.0      0.5       0.5   

   tfidf_14  tfidf_15  tfidf_16  tfidf_17  tfidf_18  tfidf_19  tfidf_20  
0       0.0       0.0  0.000000       0.0   0.57735       0.0  0.0

create a new block testing

1.   List item testing testing
2.   List item testing



clustering for reaching

#git configuration

setup

In [4]:
# --- SECTION 1: SETUP & SYNC ---
import os
from google.colab import drive, userdata

# 1. Mount Google Drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("drive already mounted")

# 2. Configuration (Ensure 'GITHUB_TOKEN' is in your Colab Secrets 🔑)
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
GITHUB_USER = "praveentecsl"
REPO_NAME = "High-Engagement-Content-Classification-model"
# Navigate to where you want to store the project in Drive
PROJECT_PATH = f"/content/drive/MyDrive/{REPO_NAME}"

# 3. Clone or Pull Logic
if not os.path.exists(PROJECT_PATH):
    # If the folder doesn't exist, go to Drive root and clone
    %cd /content/drive/MyDrive/
    !git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git
    %cd {REPO_NAME}
    print("✅ Repository Cloned for the first time.")
else:
    # If folder exists, go inside and pull latest changes from collaborator
    %cd {PROJECT_PATH}
    !git pull origin main
    print("✅ Repository updated with latest changes (Git Pull).")

# 4. Identity Configuration for Commits
!git config --global user.email "your-email@example.com"
!git config --global user.name "Praveen"

drive already mounted
/content/drive/MyDrive/High-Engagement-Content-Classification-model
From https://github.com/praveentecsl/High-Engagement-Content-Classification-model
 * branch            main       -> FETCH_HEAD
Already up to date.
✅ Repository updated with latest changes (Git Pull).


push code

In [ ]:
# --- SECTION: SAVE TO GITHUB ---
# Move to the project directory just in case
%cd {PROJECT_PATH}

# Add all changes (CSV files, plots, and the notebook itself)
!git add .

# Commit with a message
!git commit -m "testing with the file in the repo"

# Push to GitHub
!git push https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git main
print("✅ Changes pushed to GitHub successfully.")

/content/drive/MyDrive/High-Engagement-Content-Classification-model
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
✅ Changes pushed to GitHub successfully.


In [5]:
!pwd

/content/drive/MyDrive/High-Engagement-Content-Classification-model


In [8]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   SocialMedia_engagement.ipynb



In [7]:
!git add .

In [9]:
!git commit -m"move the file from colab notebooks,to repo"

[main 6f900b0] move the file from colab notebooks,to repo
 1 file changed, 1 insertion(+), 264 deletions(-)
 rewrite SocialMedia_engagement.ipynb (100%)


In [10]:
!git push https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git main
print("✅ Changes pushed to GitHub successfully.")

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.80 KiB | 184.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/praveentecsl/High-Engagement-Content-Classification-model.git
   b871caf..6f900b0  main -> main
✅ Changes pushed to GitHub successfully.
